In [85]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
import torch
import scib_metrics
from lightning.pytorch.callbacks import ModelCheckpoint
import copy
#%matplotlib inline
import sys
sys.path.append('..')
from functions import *
from SCVIModelCheckpoint import SCVIModelCheckpoint

In [90]:
adata = anndata.read_h5ad(filename="../data/adataImmHum4.h5ad")
ks = [2, 5, 10, 50, 100, 200, 500]
print(adata)

AnnData object with n_obs × n_vars = 33506 × 4000
    obs: 'batch', 'chemistry', 'data_type', 'dpt_pseudotime', 'final_annotation', 'mt_frac', 'n_counts', 'n_genes', 'sample_ID', 'size_factors', 'species', 'study', 'tissue'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'pca'
    obsm: 'Scanorama', 'Unintegrated', 'X_pca'
    varm: 'PCs'
    layers: 'counts'


In [91]:
nindex = np.random.permutation(adata.obs.index)
adata = adata[nindex,:]

In [92]:
train_adata = adata[:int(33506*0.95)]
test_adata = adata[int(33506*0.95):]
#scvi.model.SCVI.setup_anndata(test_adata, layer="counts",batch_key="batch")

In [93]:
def hyperTraining2(adata, prior, hyperpar, ks, n_epochs_kl_warmup=300, max_epochs=400, beta = 5, early_stopping=False):
    adatac = adata.copy()
    keys = []
    vaes = []
    for k in ks:
        adataPrior, vae = trainModel(adata, prior, beta, n_epochs_kl_warmup=n_epochs_kl_warmup,max_epochs=max_epochs,early_stopping=early_stopping,prior_kwargs={hyperpar:k},log=True,logname=f"hyperTrainingMGk={k}")
        keys.append(f"scVI_k={k}")
        adatac.obsm[f"scVI_k={k}"] = vae.get_latent_representation()
        vaes.append(vae)
    return adatac, keys, vaes

In [95]:
adataMG, keysMG, vaes = hyperTraining2(train_adata, "mixofgaus", "k", ks)
bmMG = runBenchmark(adataMG, keysMG, nmi_ari_cluster_labels_leiden=True, nmi_ari_cluster_labels_kmeans = True, silhouette_label=True, silhouette_batch = True, kbet_per_label=True, label_key='final_annotation')
results = bmMG.get_results(min_max_scale=False)
#results.to_csv("results/parameter_optimization_MG_2.csv")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [17:32<00:00,  2.35s/it, v_num=13, train_loss_step=1.65e+3, train_loss_epoch=1.47e+3]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [17:32<00:00,  2.63s/it, v_num=13, train_loss_step=1.65e+3, train_loss_epoch=1.47e+3]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 80/400:  20%|█▉        | 79/400 [03:14<13:39,  2.55s/it, v_num=5, train_loss_step=1.62e+3, train_loss_epoch=1.45e+3]

In [ ]:
results.to_csv("results/parameter_optimization_MG_3.csv")

In [ ]:
lls = []
for i in range(len(ks)):
    lls.append(vaes[i].get_marginal_ll(test_adata))
results["loglik"] = lls
results["k"] = ks
results.to_csv("results/parameter_optimization_MG_3.csv")

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


type: "['Metric Type'] not found in axis"

In [ ]:
plot = sns.relplot(data = results, x="loglik", y="Total")
for i in range(results.shape[0]):
    plt.annotate(results["k"][i],(results["loglik"][i],results["Total"][i]+0.001))
plot.set(title="Parameter Tuning")
plot.savefig("plots/prior_optimization_2")

In [83]:
results = pd.read_csv("results/parameter_optimization_MG_2.csv")
results = results.set_index(results["Embedding"]).drop("Metric Type")

In [94]:
vaes = []
for k in ks:
    vaes.append(scvi.model.SCVI.load(f"models/Hyper_Exp/HyperExp_mixofgaus_{k}"))

INFO     File models/Hyper_Exp/HyperExp_mixofgaus_2/model.pt already downloaded                                    
INFO     File models/Hyper_Exp/HyperExp_mixofgaus_5/model.pt already downloaded                                    
INFO     File models/Hyper_Exp/HyperExp_mixofgaus_10/model.pt already downloaded                                   
INFO     File models/Hyper_Exp/HyperExp_mixofgaus_50/model.pt already downloaded                                   
INFO     File models/Hyper_Exp/HyperExp_mixofgaus_100/model.pt already downloaded                                  
INFO     File models/Hyper_Exp/HyperExp_mixofgaus_200/model.pt already downloaded                                  
INFO     File models/Hyper_Exp/HyperExp_mixofgaus_500/model.pt already downloaded                                  


KeyboardInterrupt: 